In [14]:
import pandas as pd
import json
from datetime import datetime
import os

In [15]:
class my_dictionary(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict() 
          
    # Function to add key:value 
    def add(self, key, value): 
        self[key] = value 

# cleaning data

In [16]:
import re
df = pd.read_csv('../csvFiles/goodreadersRowfile.csv')
dfwin = pd.read_csv('../AwardDates.csv')
data = dict()
with open('../RowData/dates.json') as json_file:
    data = json.load(json_file)
for d in data: 
    date = d['date']
    result = re.search('Published(.*)', date)
    partial = result.group(1)
    if('by' in partial) :
        partial = re.search('(.*)by', partial)
        d['date'] = partial.group(1).lstrip().rstrip()
    else:
        d['date'] = partial.lstrip().rstrip()

with open('../RowData/cleanPublishDay.json', 'w') as new:
    json.dump(data, new)

 calculating dates and matching isbn

In [17]:
data = dict()
with open('../RowData/cleanPublishDay.json') as json_file:
    data = json.load(json_file)
mapUrlDate = my_dictionary()
mapUrlIsbn = my_dictionary()
isbnList = list()
for d in data:

    ds = str(d['date']).split()
    if len(ds)>1:
        ds[1]=ds[1].replace('th','').replace('st','').replace('rd','').replace('nd','')
        d['date'] = ''.join(str(e)+' ' for e in ds)
    if len(ds)>3:
         d['date'] = ''.join([ds[0],' ',ds[1],' ',ds[2],' '])
    try:
        d['date']=datetime.strptime(d['date'], '%B %d %Y ')
    except ValueError :
        try:
           d['date']= datetime.strptime(d['date'], '%B %Y ')
        except ValueError :
           d['date']= datetime.strptime(d['date'], '%Y')
    if(df[df['url']==d['url']].empty):
        continue
    year=df[df['url']==d['url']]['year'].iloc[0]
    win=datetime.strptime(dfwin[dfwin['year'] ==year]['Winners Announced'].iloc[0],'%m/%d/%Y')
    if((win -  d['date']).days<1):
        d['date']=(win -  d['date']).days + 365
    else:
        d['date']=(win -  d['date']).days 
    mapUrlDate.add(d['url'], d['date'])
    mapUrlIsbn.add(d['url'],d['isbn'])
    isbnList.append(d['isbn'])
    


# merging data and adding 'times_appred_NYTime','avgrank_NYTime','bestRank','listnames' to are coulems

In [18]:
dfUD=pd.DataFrame(mapUrlDate.items(), columns=['url', 'daysToContest'])
dfUI = pd.DataFrame(mapUrlIsbn.items(), columns=['url', 'isbn'])
df.merge(dfUD,on='url', how='left').merge(dfUI,on='url', how='left').to_csv("../csvFiles/goodreadsfinaldata.csv")


adding newyork times data 

In [19]:
df = pd.read_csv('../csvFiles/goodreadsfinaldata.csv')
df = df.reindex(df.columns.tolist() + ['times_appred_NYTime','avgrank_NYTime','bestRank','listnames'], axis=1)


In [20]:
def errorhandler_isbn(isbn):
    df.loc[df['isbn']==isbn,['times_appred_NYTime']] = 0
    df.loc[df['isbn']==isbn,['avgrank_NYTime']] =float('inf')
    df.loc[df['isbn']==isbn,['bestRank']] = float('inf')
    df.loc[df['isbn']==isbn,['listnames']] = 0
def nans(df): return df[df.isnull().any(axis=1)]

reading newYork Data and adding to feilds

In [21]:
newYork_dir = "../newYork Data/"
data = dict()
data1 = dict()
for filename in os.listdir(newYork_dir):
    if filename.endswith(".json") and filename.startswith("res") and not(filename.endswith("10.json")): 
        cmplt_path = os.path.join(newYork_dir, filename)
        with open(cmplt_path) as json_file :
            new_data = json.load(json_file)
            data.update(new_data)
    else:
        continue

print(len(data))
for i in isbnList:
    try:
        if d[i]['num_results']==0:
                errorhandler_isbn(i)
                continue
        for j in data[i]['results']:
            bestrank=float('inf')
            num=0
            count = 0 
            listname = set()
            if len(j['ranks_history'])==0:
                errorhandler_isbn(i)
                continue
            year = df[df['isbn']==i]['year']
            win=datetime.strptime(dfwin[dfwin['year'] ==year]['Winners Announced'].iloc[0],'%m/%d/%Y')
            for z in j['ranks_history']:
                if(datetime.strptime(z['published_date'], '%Y-%m-%d')<win):
                    num+= z['rank']
                    listname.add(z['list_name'])
                    if bestrank>z['rank']:
                        bestrank =z['rank']
                    count  = count + 1
            num=num/count
            df.loc[df['isbn']==i,['times_appred_NYTime']] = count
            df.loc[df['isbn']==i,['avgrank_NYTime']] =num
            df.loc[df['isbn']==i,['bestRank']] = bestrank
            df.loc[df['isbn']==i,['listnames']] = len(listname)
    except KeyError :
        errorhandler_isbn(i)
        continue


3350


saving data

In [22]:
df.to_csv("../csvFiles/withNYdata.csv")

cheak nall values

In [23]:
df = pd.read_csv('../csvFiles/withNYdata.csv')



read data from json 11 with different struct and saving to csv

In [27]:
with open('../newYork Data/res10.json') as json_file:
    data = json.load(json_file)
for i in data:
    if data[i]['num_results']==0:
        errorhandler_isbn(i)
        continue
    for j in data[i]['results']:
        bestrank=float('inf')
        num=0
        listname = set()
        if len(j['ranks_history'])==0:
                errorhandler_isbn(i)
                continue
        year = df[df['url']==i]['year']
        for k in year:
            win=datetime.strptime(dfwin[dfwin['year'] ==k]['Winners Announced'].iloc[0],'%m/%d/%Y')
            for z in j['ranks_history']:
                if(datetime.strptime(z['published_date'], '%Y-%m-%d')<win):
                    num+= z['rank']
                    listname.add(z['list_name'])
                    if bestrank>z['rank']:
                        bestrank =z['rank']
                    count  = count + 1
            num=num/count
            df.loc[(df['url']==i)&(df['year']==k),['times_appred_NYTime']] = count
            df.loc[(df['url']==i)&(df['year']==k),['avgrank_NYTime']] =num
            df.loc[(df['url']==i)&(df['year']==k),['bestRank']] = bestrank
            df.loc[(df['url']==i)&(df['year']==k),['listnames']] = str(len(listname))
df.to_csv("../csvFiles/withNYdata.csv")


saving nulls in file

In [560]:
def nans(df): return df[df.isnull().any(axis=1)]
nans(df).to_csv("nulls.csv")

